In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from mpmath import besseljzero
from mpmath import *
mp.dps = 5; mp.pretty = True

from scipy.special import gamma
from scipy.special import jv, iv
from scipy.optimize import minimize
from scipy.interpolate import interp1d

from scipy.stats import pearsonr
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
def simulate_HSDM_2D(a, mu, ndt, sigma=1, dt=0.001):
    x = np.zeros(mu.shape)
    
    rt = 0
    
    while np.linalg.norm(x, 2) < a(rt):
        x += mu*dt + sigma*np.sqrt(dt)*np.random.normal(0, 1, mu.shape)
        rt += dt
    
    theta = np.arctan2(x[1], x[0])   
    
    return ndt+rt, theta

In [4]:
def series_bessel_fpt(t, a=1, sigma=1, nu=0, n=100):
    zeros = np.asarray([float(besseljzero(nu, i+1)) for i in range(n)])
    fpt = np.zeros(t.shape)
    
    for i in range(t.shape[0]):
        series = np.sum((zeros**(nu+1)/jv(nu+1, zeros)) * np.exp(-(zeros**2 * sigma**2)/(2*a**2)*t[i]))
        fpt[i] = sigma**2/(2**nu * a**2 * gamma(nu + 1)) * series
        
    return interp1d(t, fpt)

In [5]:
def HSDM_2D_likelihood(prms, RT, Theta, N_series):
    a =  prms[0]
    ndt = prms[1]
    mu = np.array([prms[2], prms[3]])
    
    tt = np.arange(0.001, max(RT)+0.02, 0.02)
    fpt = series_bessel_fpt(tt, a, sigma=1, nu=(mu.shape[0]-2)/2, n=N_series)
    
    log_lik = 0
    for i in range(len(RT)):
        rt, theta = RT[i], Theta[i]
        if rt - ndt > 0.001:
            mu_dot_x0 = mu[0]*np.cos(theta)
            mu_dot_x1 = mu[1]*np.sin(theta)
            term1 = prms[0] * (mu_dot_x0 + mu_dot_x1)
            term2 = 0.5 * np.linalg.norm(mu, 2)**2 * (rt - ndt)
            
            density = np.exp(term1 - term2) * fpt(rt - ndt)
            
            if 0.1**14 < density:
                log_lik += -np.log(density)
            else:
                log_lik += -np.log(0.1**14)
        else:
            log_lik += -np.log(0.1**14)
        
    return log_lik

In [6]:
recovery_df = {'threshold_true': [],
               'threshold_estimate': [],
               'ndt_true': [],
               'ndt_estimate': [],
               'mu1_true': [],
               'mu1_estimate': [],
               'mu2_true': [],
               'mu2_estimate': []}

min_threshold = 0.5
max_threshold = 5

min_ndt = 0.1
max_ndt = 1

min_mu = -3.5
max_mu = 3.5

N_series = 50

In [7]:
for n in tqdm(range(150)):
    threshold = np.random.uniform(min_threshold, max_threshold)
    a = lambda t: threshold
    ndt = np.random.uniform(min_ndt, max_ndt)
    mu = np.array([np.random.uniform(min_mu, max_mu), 
                   np.random.uniform(min_mu, max_mu)])
    
    
    recovery_df['threshold_true'].append(threshold)
    recovery_df['ndt_true'].append(ndt)
    recovery_df['mu1_true'].append(mu[0])
    recovery_df['mu2_true'].append(mu[1])
    
    RT = []
    Theta = []
    
    for i in range(250):
        rt, theta = simulate_HSDM_2D(a, mu, ndt)
        RT.append(rt)
        Theta.append(theta)
    
    min_ans = minimize(HSDM_2D_likelihood,
                       args=(RT, Theta, N_series), 
                       x0=np.array([np.random.uniform(min_threshold, max_threshold),
                                    np.random.uniform(min_ndt, max_ndt), 
                                    np.random.uniform(min_mu, max_mu),
                                    np.random.uniform(min_mu, max_mu)]),
                       method='Nelder-Mead', 
                       bounds=[(min_threshold, max_threshold), (min_ndt, max_ndt),
                               (min_mu, max_mu), (min_mu, max_mu)])
    
    recovery_df['threshold_estimate'].append(min_ans.x[0])
    recovery_df['ndt_estimate'].append(min_ans.x[1])
    recovery_df['mu1_estimate'].append(min_ans.x[2])
    recovery_df['mu2_estimate'].append(min_ans.x[3])
    
recovery_df = pd.DataFrame(recovery_df)

100%|█████████████████████████████████████████████████| 150/150 [24:41<00:00,  9.88s/it]


In [8]:
recovery_df

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate
0,3.182011,2.934097,0.317115,0.343379,2.092831,2.084045,-2.324615,-2.180819
1,1.548401,1.666265,0.598879,0.592602,-3.380400,-3.499997,3.190223,3.393303
2,3.427770,3.637961,0.150487,0.100000,-1.805362,-1.849099,0.079265,0.017941
3,1.174267,1.249938,0.458236,0.443013,-1.705652,-1.552498,-3.352291,-3.287196
4,4.105426,4.035158,0.963787,1.000000,0.558629,0.473204,1.978427,1.970363
...,...,...,...,...,...,...,...,...
145,1.500333,1.607194,0.999988,1.000000,-0.493551,-0.551426,1.216092,1.240960
146,1.441672,1.538787,0.821760,0.811615,2.675752,2.850055,1.001292,0.910524
147,4.867785,5.000000,0.355325,1.000000,2.136057,3.230839,-1.729586,-2.875260
148,1.727893,1.687324,0.659937,0.695949,-0.273650,-0.323175,-0.067974,-0.070529


In [9]:
recovery_df.corr()

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate
threshold_true,1.000000,0.821091,0.014279,0.079191,-0.056105,-0.053095,-0.000094,-0.012802
threshold_estimate,0.821091,1.000000,-0.045947,-0.135505,-0.122551,-0.075161,-0.051365,-0.074839
ndt_true,0.014279,-0.045947,1.000000,0.789567,-0.057799,-0.033034,0.069075,0.056280
ndt_estimate,0.079191,-0.135505,0.789567,1.000000,-0.051820,-0.054487,0.020042,0.035528
mu1_true,-0.056105,-0.122551,-0.057799,-0.051820,1.000000,0.939661,0.059417,0.050872
mu1_estimate,-0.053095,-0.075161,-0.033034,-0.054487,0.939661,1.000000,0.074419,0.061045
mu2_true,-0.000094,-0.051365,0.069075,0.020042,0.059417,0.074419,1.000000,0.974161
mu2_estimate,-0.012802,-0.074839,0.056280,0.035528,0.050872,0.061045,0.974161,1.000000


In [10]:
recovery_df.to_csv('Series_2d_recovery_5digit.csv'.format(N_series))